In [2]:
import numpy as np

def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = np.array([[0, -p[2], p[1]],
                      [p[2], 0, -p[0]],
                      [-p[1], p[0], 0]])
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                         np.hstack((p_mat @ R, R))))
    return Adjoint

def skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                         [w[2], 0, -w[0]],
                         [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def screw(w, q):
    w_mat = np.array([[0, -w[2], w[1]],
                      [w[2], 0, -w[0]],
                      [-w[1], w[0], 0]], dtype = object)
    v = -np.cross(w, q)
    screw = np.vstack((np.hstack((w_mat, v.reshape(-1, 1))), np.zeros((1, 4))))
    return screw


In [3]:
import numpy as np
import time

from scipy.linalg import expm
from scipy.linalg import logm

q = np.array([0, 0.2004, 0.0068])

M = np.vstack([np.hstack([np.eye(3), q.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])

w1 = np.array([0, 0, 1])
w2 = np.array([1, 0, 0])
w3 = np.array([1, 0, 0])
w4 = np.array([1, 0, 0])
w5 = np.array([0, 0, 1])

q1 = np.array([0, 0, 0])
q2 = np.array([0, 0, 0.0701])
q3 = np.array([0, 0.1036, 0.0701])
q4 = np.array([0, 0.2004, 0.0701])
q5 = np.array([0, 0.2004, 0.0391])

v1 = -np.cross(w1, q1)
v2 = -np.cross(w2, q2)
v3 = -np.cross(w3, q3)
v4 = -np.cross(w4, q4)
v5 = -np.cross(w5, q5)

S1 = screw(w1, q1)
S2 = screw(w2, q2)
S3 = screw(w3, q3)
S4 = screw(w4, q4)
S5 = screw(w5, q5)

w1 = w1.reshape(-1,1)
w2 = w2.reshape(-1,1)
w3 = w3.reshape(-1,1)
w4 = w4.reshape(-1,1)
w5 = w5.reshape(-1,1)

v1 = v1.reshape(-1,1)
v2 = v2.reshape(-1,1)
v3 = v3.reshape(-1,1)
v4 = v4.reshape(-1,1)
v5 = v5.reshape(-1,1)

# start position : [0.15, 0.07, 0.0701]
x_d = np.array([0.1, 0.12, 0.0701]) 
x_d = x_d.reshape(-1,1)
T_sd = np.vstack([np.hstack([np.eye(3), x_d]),
                  np.array([[0, 0, 0, 1]])])

# th = np.deg2rad([-86.9474, 41.3064, -44.3338, 3.0274, 86.9474])
th = np.deg2rad([-65.28392204, 47.79348963, -55.77497405, 7.98148442, 65.28392204])
    
def ctrl_all_servo(angle, s_time = 1000):
    Arm.Arm_serial_servo_write6(thd[0], thd[1], thd[2], thd[3], thd[4], 90, s_time)
    time.sleep(s_time/1000)
        
w_b = np.array([1, 1, 1])
w_b = w_b.reshape(-1,1)
v_b = np.array([1, 1, 1])
v_b = v_b.reshape(-1,1)

k = 1
dt = 2

while (np.linalg.norm(w_b) > 0.001 or np.linalg.norm(v_b) > 0.0001):


    T_sb = expm(S1 * th[0]) @ expm(S2 * th[1]) @ expm(S3 * th[2]) @ expm(S4 * th[3]) @ expm(S5 * th[4]) @ M
    T_bs = np.linalg.inv(T_sb)

    AdT_bs = Ad(T_bs)

    Js1 = np.vstack([w1, v1])
    Js2 = Ad(expm(S1 * th[0])) @ np.vstack([w2, v2])
    Js3 = Ad(expm(S1 * th[0]) @ expm(S2 * th[1])) @ np.vstack([w3, v3])
    Js4 = Ad(expm(S1 * th[0]) @ expm(S2 * th[1]) @ expm(S3 * th[2])) @ np.vstack([w4, v4])
    Js5 = Ad(expm(S1 * th[0]) @ expm(S2 * th[1]) @ expm(S3 * th[2]) @ expm(S4 * th[3])) @ np.vstack([w5, v5])
    Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
    Jb = AdT_bs @ Js

    T_bd = T_bs @ T_sd
    V_bmat = logm(T_bd) * 0.1
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    v_b = V_bmat[:3, 3]
    V_b = np.hstack([w_b, v_b])

    gamma = 0.2 
    th_home = np.array([0, 0, 0, 0, 0])
    dth = np.linalg.pinv(Jb) @ V_b + gamma * (np.eye(5) - np.linalg.pinv(Jb) @ Jb) @ (th_home - th)

    th = th + dth*dt
    th_deg = np.rad2deg(th)
    
    thd = [th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90]
    
    k = k + 1
    
    if k == 200:
        break

In [6]:
th_deg

array([-40.04618905,  53.64065771, -65.64484852,  12.00419081,
        40.04618905])